In [1]:
import datetime
import random
import json
import dotenv
from azure.iot.device import IoTHubDeviceClient
from azure.iot.hub import IoTHubRegistryManager
from os import getenv

In [2]:
oid = "XYZ"

In [3]:
PEDOMETER_DEVICE_CONNECTION_STRING = "HostName=my-project-iot-hub.azure-devices.net;DeviceId=2;SharedAccessKey=5IPRra4goPXr7VvEne8hcaXcNqADQJp10AIoTKDfA0Y="
pedometer_device_id = PEDOMETER_DEVICE_CONNECTION_STRING.split(";")[1].split("=")[1]

In [4]:
SCALE_DEVICE_CONNECTION_STRING = "HostName=my-project-iot-hub.azure-devices.net;DeviceId=2;SharedAccessKey=5IPRra4goPXr7VvEne8hcaXcNqADQJp10AIoTKDfA0Y="
scale_device_id = SCALE_DEVICE_CONNECTION_STRING.split(";")[1].split("=")[1]

In [5]:
dotenv.load_dotenv("./device-function-app/.env")
# pRINT OUT THE ENVIRONMENT VARIABLES
getenv("IOT_HUB_CONNECTION_STRING")

'HostName=my-project-iot-hub.azure-devices.net;SharedAccessKeyName=iothubowner;SharedAccessKey=qqcgU6ZJIHaw83SPpqvvomMIndFvmoirmAIoTHrYNkU='

In [6]:
registry = IoTHubRegistryManager.from_connection_string(getenv("IOT_HUB_CONNECTION_STRING"))

In [7]:
pedometer_device_twin = registry.get_twin(pedometer_device_id)

In [ ]:
# Create an IoT Hub client
pedometer_device = IoTHubDeviceClient.create_from_connection_string(PEDOMETER_DEVICE_CONNECTION_STRING)
pedometer_device.connect()
scale_device = IoTHubDeviceClient.create_from_connection_string(SCALE_DEVICE_CONNECTION_STRING)
scale_device.connect()

Exception caught in background thread.  Unable to handle.
['azure.iot.device.common.transport_exceptions.ConnectionDroppedError: Unexpected disconnection\n']


In [9]:
from abc import ABC
class Measurement(ABC):
    def __init__(self, oid: str, datetime: str):
        self.oid = oid
        self.datetime = datetime

In [10]:
class PedometerMeasurement(Measurement):
    def __init__(self, steps, distance, datetime: datetime.datetime, oid: str, unit="30m"):
        super().__init__(oid, datetime.isoformat())
        self.steps = steps
        self.distance = distance
        self.unit = unit
    
    def __str__(self):
        return f"Device Type: Steps: {self.steps}, Distance: {self.distance}, Unit: {self.unit}, Datetime: {self.datetime}"

In [11]:
class ScaleMeasurement(Measurement):
    def __init__(self, weight, timestamp: datetime.datetime, oid: str, unit="kg"):
        super().__init__(oid, timestamp.isoformat())
        self.weight = weight
        self.unit = unit

    def __str__(self):
        return f"Device Type: Weight: {self.weight}, Unit: {self.unit}, Datetime: {self.datetime}"

In [12]:
current_time = datetime.datetime.now()
current_time -= datetime.timedelta(minutes=current_time.minute % 30, seconds=current_time.second, microseconds=current_time.microsecond)
thirty_minutes = datetime.timedelta(minutes=45) # Round to the previous whole 30 minutes
# Send pedometer measurements
for i in range(300):
    pm = PedometerMeasurement(random.randint(0, 100), random.randint(0, 100), current_time, oid)
    print(pm)
    current_time -= thirty_minutes
    message = json.dumps(pm.__dict__)
    print(message)
    pedometer_device.send_message(message)

Exception caught in background thread.  Unable to handle.
['azure.iot.device.common.transport_exceptions.ConnectionDroppedError: Unexpected disconnection\n']
Exception caught in background thread.  Unable to handle.
['azure.iot.device.common.transport_exceptions.ConnectionDroppedError: Unexpected disconnection\n']
Exception caught in background thread.  Unable to handle.
['azure.iot.device.common.transport_exceptions.ConnectionDroppedError: Unexpected disconnection\n']
Exception caught in background thread.  Unable to handle.
['azure.iot.device.common.transport_exceptions.ConnectionDroppedError: Unexpected disconnection\n']
Exception caught in background thread.  Unable to handle.
['azure.iot.device.common.transport_exceptions.ConnectionDroppedError: Unexpected disconnection\n']
Exception caught in background thread.  Unable to handle.
['azure.iot.device.common.transport_exceptions.ConnectionDroppedError: Unexpected disconnection\n']
Exception caught in background thread.  Unable to ha

Device Type: Steps: 3, Distance: 9, Unit: 30m, Datetime: 2024-02-07T11:30:00
{"oid": "XYZ", "datetime": "2024-02-07T11:30:00", "steps": 3, "distance": 9, "unit": "30m"}
Device Type: Steps: 83, Distance: 39, Unit: 30m, Datetime: 2024-02-07T10:45:00
{"oid": "XYZ", "datetime": "2024-02-07T10:45:00", "steps": 83, "distance": 39, "unit": "30m"}
Device Type: Steps: 20, Distance: 36, Unit: 30m, Datetime: 2024-02-07T10:00:00
{"oid": "XYZ", "datetime": "2024-02-07T10:00:00", "steps": 20, "distance": 36, "unit": "30m"}
Device Type: Steps: 84, Distance: 87, Unit: 30m, Datetime: 2024-02-07T09:15:00
{"oid": "XYZ", "datetime": "2024-02-07T09:15:00", "steps": 84, "distance": 87, "unit": "30m"}
Device Type: Steps: 46, Distance: 19, Unit: 30m, Datetime: 2024-02-07T08:30:00
{"oid": "XYZ", "datetime": "2024-02-07T08:30:00", "steps": 46, "distance": 19, "unit": "30m"}
Device Type: Steps: 82, Distance: 6, Unit: 30m, Datetime: 2024-02-07T07:45:00
{"oid": "XYZ", "datetime": "2024-02-07T07:45:00", "steps": 82

In [13]:
current_time = datetime.datetime.now()
current_time -= datetime.timedelta(minutes=current_time.minute % 30, seconds=current_time.second, microseconds=current_time.microsecond)
thirty_minutes = datetime.timedelta(minutes=45) # Round to the previous whole 30 minutes
# Send pedometer measurements
for i in range(300):
    pm = ScaleMeasurement(random.randint(65, 75) + random.random(), current_time, oid)
    print(pm)
    current_time -= thirty_minutes
    message = json.dumps(pm.__dict__)
    print(message)
    scale_device.send_message(message)

Device Type: Weight: 70.71915798494473, Unit: kg, Datetime: 2024-02-07T11:30:00
{"oid": "XYZ", "datetime": "2024-02-07T11:30:00", "weight": 70.71915798494473, "unit": "kg"}
Device Type: Weight: 67.74544625576772, Unit: kg, Datetime: 2024-02-07T10:45:00
{"oid": "XYZ", "datetime": "2024-02-07T10:45:00", "weight": 67.74544625576772, "unit": "kg"}
Device Type: Weight: 65.61598455774332, Unit: kg, Datetime: 2024-02-07T10:00:00
{"oid": "XYZ", "datetime": "2024-02-07T10:00:00", "weight": 65.61598455774332, "unit": "kg"}
Device Type: Weight: 72.76318633950645, Unit: kg, Datetime: 2024-02-07T09:15:00
{"oid": "XYZ", "datetime": "2024-02-07T09:15:00", "weight": 72.76318633950645, "unit": "kg"}
Device Type: Weight: 68.58444004045909, Unit: kg, Datetime: 2024-02-07T08:30:00
{"oid": "XYZ", "datetime": "2024-02-07T08:30:00", "weight": 68.58444004045909, "unit": "kg"}
Device Type: Weight: 69.37616682006471, Unit: kg, Datetime: 2024-02-07T07:45:00
{"oid": "XYZ", "datetime": "2024-02-07T07:45:00", "weig